# Object Following - Live Demo （对象跟踪-实时演示）

在这本笔记本中，我们将展示如何使用JetBot跟踪对象！我们将使用预先训练好的神经网络
这是在[COCO数据集]（http://COCO dataset.org）上训练的，用来检测90个不同的公共对象。其中包括

*人（索引 0）、杯（索引 47）*

以及许多其他（您可以查看 [此文件](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoo_complete_label_map.pbtxt) 以获取类索引的完整列表）。该模型来源于[TensorFlow对象检测API](https://github.com/TensorFlow/models/tree/master/research/object-detection)，它还为自定义任务的对象检测器培训提供实用程序！一旦模型得到训练，我们就在Jetson Nano上使用NVIDIA TensorRT对其进行优化。
这使得网络非常快速，能够在Jetson Nano上实时执行！不过，我们不会在本笔记本中详细介绍所有的培训和优化步骤。

不管怎样，我们开始吧。首先，我们要导入 ``ObjectDetector`` 类，该类使用经过预训练的 SSD 引擎。

### 单摄像机图像上的检测计算

In [ ]:
from jetbot import ObjectDetector
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

在内部，``ObjectDetector``类使用TensorRT Python API来执行我们提供的引擎。
它还负责对神经网络的输入进行预处理，以及分析检测到的对象。现在它只适用于使用 ``jetbot.ssd_tensorrt`` 包创建的引擎。
该包具有用于转换的实用程序从TensorFlow对象检测API到优化tensort引擎的模型。

接下来初始化相机，因为我们用的的探测器（detector）需要 300x300 像素的输入，所以我们将在创建相机时设置这个尺寸。

>在内部，Camera 类使用 GStreamer 来利用 Jetson Nano 的图像信号处理器（ISP），可以大大减轻CPU调整图片尺寸的压力，提升性能。

In [ ]:
from jetbot import Camera
camera = Camera.instance(width=300, height=300)

现在，让我们使用一些摄像机输入来执行我们的网络。默认情况下，``ObjectDetector`` 类需要相机生成的 ``bgr8``  格式。如果输入的格式不同，则可以重写默认的预处理函数。

In [ ]:
detections = model(camera.value)
print(detections)

如果摄像机视野中有任何COCO对象，它们现在应该存储在 ``detections`` 变量中。

### 在文本区域显示检测结果
我们将使用下面的代码打印出检测到的对象。

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

您应该看到在每个图像中检测到的每个对象的标签、置信度和边界框。在这个例子中只有一个图像（我们的相机）。
要只打印第一幅图像中检测到的第一个对象，可以调用以下命令
>如果未检测到对象，则可能会引发错误

In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

### 控制小车跟踪中心物体

现在我们希望小车跟随指定类的对象。为此，我们将执行以下操作

1. 检测与指定类匹配的对象
2. 选择离摄像机视野中心最近的对象，这是 “target” 对象
3. 引导机器人朝向目标物体，否则会漂移
4. 如果我们被障碍物挡住了，向左拐

我们还将创建一些小部件，用于控制目标对象标签、小车速度和一种“转弯增益（turn gain）”，根据目标物体之间的距离来控制机器人的转弯速度以及小车视野的中心。

首先，让我们加载避撞（collision avoidance）模型。为了方便起见，预先训练的模型存储在这个目录中

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

太好了，现在让我们初始化我们的小车，这样就可以控制马达了。

In [ ]:
from jetbot import Robot
robot = Robot()

最后，让我们显示所有控件小部件，并将网络执行功能连接到相机更新。

In [ ]:
from jetbot import bgr8_to_jpeg

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked
    
    # turn left if blocked
    if prob_blocked > 0.5:
        robot.left(0.3)
        image_widget.value = bgr8_to_jpeg(image)
        return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
        
    # otherwise go forward if no target detected
    if det is None:
        robot.forward(float(speed_widget.value))
        
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

调用下面的块将execute函数连接到每个相机帧更新。

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

令人惊叹的！如果机器人没有被阻挡，你应该看到在检测到的物体周围用蓝色画出的方框。目标对象（机器人跟随）将显示为绿色。

当探测到目标时，机器人应该转向目标。如果它被一个物体挡住了，它只会向左拐。

您可以调用下面的代码块来手动断开处理与相机的连接并停止机器人。

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()